<a href="https://colab.research.google.com/github/amanmaurya7/langchain_collab/blob/main/Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langchain-community

In [52]:
from langchain_community.chat_models import ChatPerplexity
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
import os
from google.colab import userdata

# Retrieve API key from Colab Secrets
# Ensure you have added 'Perplexity_API_TOKEN' as a secret in the Colab 'Secrets' panel (🔑 icon on the left).
if not os.environ.get("Perplexity_API_TOKEN"):
    os.environ["Perplexity_API_TOKEN"] = userdata.get('Perplexity_API_TOKEN')

llm = ChatPerplexity(model = 'sonar-pro', temperature=0.3, max_tokens = 500)

**Rule Based Agent**

In [ ]:
from inspect import currentframe
def thermostat_action(curr_temp):
  """Determine the action based on the current temperature"""
  if curr_temp < 18:
    return 'Turn on the Heater'
  elif curr_temp > 25:
    return 'Turn on the Air Cooler'
  else:
    return 'No action'

curr_temp = 21

**LLM Based Agent**

In [31]:
prompt = PromptTemplate(
    input_variables=['student_info'],
    template = """ You are Scholarship Evaluator Assisant
    to be eligible for scholarship student must:
    {student_info}
    1. Student age >= 20 years
    2. Student overall score >= 85%
    3. Particpated in any Extra Curicullar Activity

    Evaluate Eligibility cleary as Eligible or Not Eligible for Scholarship
    """
)

In [32]:
chain = prompt | llm | StrOutputParser()

In [33]:
student_info = 'Student is 22 years old, has scored 88% marks and plays football'

In [34]:
print(chain.invoke({'student_info': student_info}))

Based on the eligibility criteria you've provided, this student is **Eligible** for the scholarship.

**Eligibility Assessment:**

| Criterion | Requirement | Student Status | Result |
|-----------|-------------|-----------------|--------|
| Age | >= 20 years | 22 years old | ✓ Meets requirement |
| Academic Performance | >= 85% | 88% marks | ✓ Meets requirement |
| Extracurricular Activity | Participation required | Plays football | ✓ Meets requirement |

The student satisfies all three eligibility criteria and should proceed to the evaluation phase, where additional factors such as financial need, letters of recommendation, and alignment with the scholarship program's mission would typically be assessed[1][2].


**Hybrid Agent**

In [47]:
prompt = PromptTemplate(
    input_variable=['query'],
    template='Answer the following customer support query: {query}'
)

In [48]:
def rule_based_response(query):
  """Handles simple queries with predefined rules:"""
  rules={
      'order status':'Your order is being processed and should arrive within 3-5 business days',
      'shipping time':'shipping take 3-5 days business days, depending on your location',
      'refund policy': 'You can return items within 30 days of purchase'
  }
  query = query.lower()
  for key,response in rules.items():
    if key in query:
      return response
  return None

In [49]:
def llm_based_response(query):
  """Generates response using a langauge model fro complex queries."""
  chain = prompt | llm | StrOutputParser()
  chain.invoke({'query':query})

In [50]:
def hybrid_agent(query):
  """Hybrid Agent combines rule-based and LLM based Response"""
  response = rule_based_response(query)
  if response:
    return response
  else:
    return llm_based_response(query)

In [51]:
query = 'Order status'
response = hybrid_agent(query)
print(f"Response: {response}")

Response: Your order is being processed and should arrive within 3-5 business days


**Planner Agent**

In [53]:
template = """ You are Task Planner, Break down the task into smaller, actionable steps and output each in steps in a clear concise manner
Task : {task}

Output a step by step itinary with time and places
Steps:
"""

In [54]:
prompt = ChatPromptTemplate.from_template(template)

In [55]:
chain  = prompt | llm | StrOutputParser()

In [56]:
print(chain.invoke({'task':'Plan a 2 day trip to mumbai'}))

### Day 1: Arrival, Elephanta Caves, and South Mumbai Highlights
- **8:00 AM - 9:00 AM**: Arrive at Chhatrapati Shivaji International Airport; transfer to hotel in South Mumbai (e.g., Colaba area) for check-in and freshen up.[1][2]
- **9:30 AM - 12:00 PM**: Head to Gateway of India; take a 1-hour ferry to Elephanta Caves (UNESCO site with ancient Shiva temples).[1][2][3][4]
- **12:00 PM - 2:00 PM**: Explore Elephanta Caves with a guided tour; return by ferry.[1][3][4]
- **2:00 PM - 3:00 PM**: Lunch at Leopold Cafe or nearby eatery in Colaba (try local snacks like bhel puri).[2][3][4]
- **3:30 PM - 5:00 PM**: Visit Chhatrapati Shivaji Maharaj Terminus (CST), admire Gothic architecture (UNESCO site).[1][2][3]
- **5:30 PM - 7:30 PM**: Stroll Marine Drive (Queen's Necklace) and Chowpatty Beach for sunset views and street food.[1][2][3][4]
- **8:00 PM onwards**: Dinner at hotel or Nariman Point; overnight at hotel.[1][2]

### Day 2: Local Sightseeing, Temples, and Markets
- **8:00 AM - 9:30